In [18]:
import numpy as np
import gradio as gr
import math

#### Check audio output with non-streaming mode

In [ ]:
def get_bip_bip(total_duration=5, chunk_duration=None, bip_duration=1.25, sample_rate=32000):
    """Produces a series of 'bip bip' sounds at a given frequency."""
    chunk_duration = chunk_duration if chunk_duration is not None else total_duration * sample_rate // 5
    timesteps = np.arange(int(total_duration * sample_rate)) / sample_rate
    wav = np.cos(2 * math.pi * 440 * timesteps)
    time_period = (timesteps % (2 * bip_duration)) / (2 * bip_duration)
    envelope = time_period >= 0.5
    waveform = wav * envelope
    return waveform

def get_bip_bip_static(total_duration=5, chunk_duration=None, bip_duration=0.125, sample_rate=32000):
    """Returns the entire waveform in one go."""
    waveform = get_bip_bip(total_duration=total_duration, chunk_duration=chunk_duration, bip_duration=bip_duration, sample_rate=sample_rate)
    return gr.make_waveform((sample_rate, waveform))

In [ ]:
# check our static bip bip works (spoiler: it does!)
with gr.Blocks() as demo:
    with gr.Column():
        total_duration = gr.Slider(10, 30, value=15, step=5, label="Audio length in s")
        audio_input_btn = gr.Button("Return array")
        audio_output = gr.Audio(streaming=False, autoplay=True)
    
        audio_input_btn.click(get_bip_bip_static, inputs=[total_duration], outputs=audio_output)

demo.queue().launch()

#### Check audio output with streaming mode

In [19]:
target_dtype = np.int16
max_range = np.iinfo(target_dtype).max

def get_bip_bip_streaming(total_duration=5, chunk_duration=None, bip_duration=0.125, sample_rate=32000):
    """Returns the waveform in chunks of specified duration."""
    waveform = get_bip_bip(total_duration=total_duration, chunk_duration=chunk_duration, bip_duration=bip_duration, sample_rate=sample_rate)
    i = 0
    while chunk_duration*i < len(waveform):
        chunk = waveform[chunk_duration*i:chunk_duration*(i+1)]
        chunk = (chunk * max_range).astype(np.int16)
        yield (sample_rate, chunk)
        time.sleep(chunk_duration *0.9)
        i += 1

In [20]:
# check our streaming bip bip works - currently errors out
with gr.Blocks() as demo:
    with gr.Column():
        total_duration = gr.Slider(10, 30, value=15, step=5, label="Audio length in s")
        chunk_duration = gr.Slider(2, 10, value=2, step=2, label="Streaming interval in s")
        stream_input_btn = gr.Button("Stream as array")
        stream_output = gr.Audio(streaming=True, type="numpy", autoplay=True)
    
        stream_input_btn.click(get_bip_bip_streaming, inputs=[total_duration, chunk_duration], outputs=stream_output)

demo.queue().launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
